In [ ]:
from pysam import VariantFile

In [ ]:
childVCF = VariantFile("HG002-NA24385-50x_filtered.vcf")
fatherVCF = VariantFile("HG003.hs37d5.60x.1.converted_filtered.vcf")
motherVCF = VariantFile("HG004.hs37d5.60x.1.converted_filtered.vcf")

In [ ]:
def findMaxChromPos(VCFs):
    maxPos = 0
    for vcf in VCFs:  
        for rec in vcf.fetch():
            maxPos = max(maxPos, rec.pos)
    return maxPos, len(str(abs(maxPos)))

In [ ]:
def checkPosIntersection(childVariant, parentVariant):  
    
    if childVariant.ref != parentVariant.ref:
        return False, None
    
    #for X or Y chromosomes, return child if ref, alts and pos is the same
    if childVariant.chrom == "chrX" or childVariant.chrom == "chrY":
        if childVariant.alts == parentVariant.alts:
            return True, childVariant
        else:
            return False, None            
    
    childGT = next(childVariant.samples.itervalues())['GT']
    parentGT = next(parentVariant.samples.itervalues())['GT']
    gt01 = (0,1)
    gt11 = (1,1)
    gt12 = (1,2)
    childALT = childVariant.alts
    parentALT = parentVariant.alts
    
    if parentGT == gt01:
        # parent has 0/1 genotype, child has 0/1 genotype
        # if child and parent have the same REF and ALT -> return the variant
        if childGT == gt01:
            if childALT[0] == parentALT[0]:
                return True, childVariant
        # parent has 0/1 genotype, child has 1/1 genotype
        # if child and parent have the same REF and ALT -> child variant with 0/1 gt goes into intersection
        elif childGT == gt11:
            if childALT[0] == parentALT[0]:
                next(childVariant.samples.itervalues())['GT'] = (0,1)
                return True, childVariant
        # parent has 0/1 genotype, child has 1/2 genotype
        # if one of child's alts is the same as parent's alt-> return the variant
        else :
            if childALT[0] == parentALT[0] or childALT[1] == parentALT[0]:
                next(childVariant.samples.itervalues())['GT'] = (0,1)
                childVariant.alts = parentVariant.alts
                return True, childVariant
    elif parentGT == gt11:
        # parent has 1/1 genotype, child has 0/1 genotype
        # if child's alt is the same as parent's alt -> return the variant
        if childGT == gt01:
            if childALT[0] == parentALT[0]:
                return True, childVariant
        # parent has 1/1 genotype, child has 1/1 genotype
        # if child's alt is the same as parent's alt-> return the variant
        elif childGT == gt11:
            if childALT[0] == parentALT[0]:
                return True, childVariant
        # parent has 1/1 genotype, child has 1/2 genotype
        # if one of child's alts is the same as parent's-> return the variant
        else :
            if childALT[0] == parentALT[0] or childALT[1] == parentALT[0]:
                next(childVariant.samples.itervalues())['GT'] = (0,1)
                childVariant.alts = parentVariant.alts
                return True, childVariant
    else :
        # parent has 1/2 genotype, child has 0/1 genotype
        # if child's alt is the same as one of parent's alts-> return the variant
        if childGT == gt01:
            if childALT[0] == parentALT[0] or childALT[0] == parentALT[1]:
                next(childVariant.samples.itervalues())['GT'] = (0,1)
                return True, childVariant
        #parent has 1/2 genotype, child has 1/1 genotype
        elif childGT == gt11:
            if childALT[0] == parentALT[0] or childALT[0] == parentALT[1]:
                next(childVariant.samples.itervalues())['GT'] = (0,1)
                return True, childVariant
        #parent has 1/2 genotype, child has 1/2 genotype
        else :
            #if child has both variations as parent
            if childALT[0] == parentALT[0] and childALT[1] == parentALT[1] or childALT[0] == parentALT[1] and childALT[1] == parentALT[0]:
                return True, childVariant
            elif childALT[0] == parentALT[0] or childALT[0] == parentALT[1]:
                next(childVariant.samples.itervalues())['GT'] = (0,1)
                childVariant.alts = (childALT[0],)
                return True, childVariant
            elif childALT[1] == parentALT[0] or childALT[1] == parentALT[1]:
                next(childVariant.samples.itervalues())['GT'] = (0,1)
                childVariant.alts = (childALT[1],)
                return True, childVariant
            
    return False, None

In [ ]:
def skipXYChrom(variant, vcf, useXY, isMother):
    if variant is None:
        return None
    if variant.chrom == "chrX" or variant.chrom == "chrY": 
        if not useXY: 
            while variant.chrom != "chrM":
                variant = next(vcf, None)
            return variant
        else: 
            if isMother:
                while variant.chrom == "chrY":
                    variant = next(vcf, None)
                return variant
            else:
                while variant.chrom == "chrX":
                    variant = next(vcf, None)
                return variant  
    else:
        return variant

In [ ]:
def findIntersection(childVCF, parentVCF, outputFileName, useXY, isMother, intersectionType, filterPass = True):
    #useXY -> if true, intersection is done on all chromosomes, if false, X and Y are skipped
    #isMother -> should be true if intersecting child with mother, false if intersecting child with father
    #intersectionType -> if true, when intersecting with mother we skip Y, when intersecting with father we skip X,
    # if false, we treat X and Y as any other chromosome
    #filterPass -> if true, check intersection only for records with PASS in filter column, if false, check all
    
    # check filename extension -----------------------
    outputVCF = VariantFile(outputFileName, 'w', header=childVCF.header)
    chrom = ""
    
    for (childVariant, parentVariant) in zip(childVCF.fetch(),parentVCF.fetch()):
        if intersectionType:            
            childVariant = skipXYChrom(childVariant, childVCF, useXY, isMother)
            parentVariant = skipXYChrom(parentVariant, parentVCF, useXY, isMother)
        #find the same variant position on the same chromosome for both
        while not (parentVariant is None or childVariant is None) and (childVariant.pos != parentVariant.pos or childVariant.chrom != parentVariant.chrom):
            while not (parentVariant is None or childVariant is None) and (childVariant.pos > parentVariant.pos):
                if childVariant.chrom != parentVariant.chrom:
                    childVariant = next(childVCF, None)
                    if intersectionType: childVariant = skipXYChrom(childVariant, childVCF, useXY, isMother)
                else:
                    parentVariant = next(parentVCF, None)
                    if intersectionType: parentVariant = skipXYChrom(parentVariant, parentVCF, useXY, isMother)
            while not (parentVariant is None or childVariant is None) and (childVariant.pos < parentVariant.pos):
                if childVariant.chrom != parentVariant.chrom:
                    parentVariant = next(parentVCF, None)
                    if intersectionType: parentVariant = skipXYChrom(parentVariant, parentVCF, useXY, isMother)
                else:
                    childVariant = next(childVCF, None)
                    if intersectionType: childVariant = skipXYChrom(childVariant, childVCF, useXY, isMother)
        
        #if one reaches end after chromosome M, iterate through the other
        if parentVariant is None:
            while not (childVariant is None):
                childVariant = next(childVCF, None)
        elif childVariant is None:
            while not (parentVariant is None):
                parentVariant = next(parentVCF, None)
        #child variant position and parent variant position are matched
        else:   
            if filterPass and (next(childVariant.filter.iterkeys()) != 'PASS' or next(parentVariant.filter.iterkeys()) != 'PASS'): 
                continue
           
            condition, variant = checkPosIntersection(childVariant, parentVariant)
            if condition: 
                outputVCF.write(variant)
            
            #printing
            if chrom != childVariant.chrom:
                chrom = childVariant.chrom
                print("Intersection on "+chrom)
            
    outputVCF.close()
    #return outputVCF

In [ ]:
def correctIntersectionToUnion(variant):
    #for X or Y chromosomes keep the 1/1 or 1/2 format
    if variant.chrom == "chrX" or variant.chrom == "chrY":
        return variant
    
    gt = next(variant.samples.itervalues())['GT']  
    
    #for somatic chromosomes, child can get only 1 chromsome from each parent, so reduce 1/1 or 1/2 to 0/1
    if gt == (1,1):
        next(variant.samples.itervalues())['GT'] = (0,1)
    elif gt == (1,2) :
        next(variant.samples.itervalues())['GT'] = (0,1)
        variant.alts = (variant.alts[0],)
        
    return variant

In [ ]:
def findUnion(VCF1, VCF2, outputFileName):
    #unionType -> same as intersectionType - if true, child-mother has only X and child-father only Y,
    #if false, both mother and father have intersections on both X and Y
    chrom = ""
    # check filename extension -----------------------
    outputVCF = VariantFile(outputFileName, 'w', header=VCF1.header)
    for (variant1, variant2) in zip(VCF1.fetch(),VCF2.fetch()):
        #find the same variant position on the same chromosome for both
        while not (variant2 is None or variant1 is None) and (variant1.pos != variant2.pos or variant1.chrom != variant2.chrom):
            while not (variant2 is None or variant1 is None) and (variant1.pos > variant2.pos):
                if variant1.chrom != variant2.chrom:
                    if variant1.chrom == "chrY" and variant2.chrom == "chrX":
                        outputVCF.write(correctIntersectionToUnion(variant2)) 
                        variant2 = next(VCF2, None) 
                    else:
                        #variant2 on that position is gt 0/0
                        outputVCF.write(correctIntersectionToUnion(variant1)) 
                        variant1 = next(VCF1, None) 
                else:  
                    #variant1 on that position is gt 0/0
                    outputVCF.write(correctIntersectionToUnion(variant2))
                    variant2 = next(VCF2, None)
            while not (variant2 is None or variant1 is None) and (variant1.pos < variant2.pos):
                if variant1.chrom != variant2.chrom:
                    if variant1.chrom == "chrX" and variant2.chrom == "chrY":
                        outputVCF.write(correctIntersectionToUnion(variant1)) 
                        variant1 = next(VCF1, None) 
                    else:
                        #variant1 on that position is gt 0/0
                        outputVCF.write(correctIntersectionToUnion(variant2))
                        variant2 = next(VCF2, None)
                else:
                    #variant2 on that position is gt 0/0
                    outputVCF.write(correctIntersectionToUnion(variant1))
                    variant1 = next(VCF1, None) 
        
        if variant1 is None:
            while not (variant2 is None):
                outputVCF.write(correctIntersectionToUnion(variant2))
                variant2 = next(VCF2, None)
        elif variant2 is None:
            while not (variant1 is None):
                outputVCF.write(correctIntersectionToUnion(variant1))
                variant1 = next(VCF1, None)
        else: 
            #variant positions are matched
            
            #printing
            if chrom != variant1.chrom:
                chrom = variant1.chrom
                print("union on "+chrom)
                
            #if position matched on X or Y chromosomes - that's the same (child's) variation
            if variant1.chrom == "chrX" or variant1.chrom == "chrY":
                outputVCF.write(variant1)
                continue
            
            #if position matched on somatic chromosomes
            gt1 = next(variant1.samples.itervalues())['GT']
            gt2 = next(variant2.samples.itervalues())['GT']

            if gt1 == (0,1) and gt2 == (0,1):
                if variant1.alts == variant2.alts:
                    next(variant1.samples.itervalues())['GT'] = (1,1)
                else:
                    variant1.alts = (variant1.alts[0],variant2.alts[0])
                    next(variant1.samples.itervalues())['GT'] = (1,2)
                outputVCF.write(variant1)
            elif gt1 == (1,1) and gt2 == (1,1) or gt1 == (1,2) and gt2 == (1,2) or (gt2 == (0,1) and (gt1 == (1,1) or gt1 == (1,2))):
                outputVCF.write(variant1)
            elif gt1 == (0,1) and (gt2 == (1,1) or gt2 == (1,2)):
                outputVCF.write(variant2)
            else:
                print("error "+str(gt1)+" "+str(gt2))
    
    outputVCF.close()

In [ ]:
def findDisjunction(unionVCF, childVCF, outputFileName, filterPass = True):
    chrom = ""
    # check filename extension -----------------------
    outputVCF = VariantFile(outputFileName, 'w', header=childVCF.header)
    for (unionVariant, childVariant) in zip(unionVCF.fetch(),childVCF.fetch()):
        #find the same variant position on the same chromosome for both
        while not (childVariant is None or unionVariant is None) and (unionVariant.pos != childVariant.pos or unionVariant.chrom != childVariant.chrom):
            while not (childVariant is None or unionVariant is None) and (unionVariant.pos > childVariant.pos):
                if unionVariant.chrom != childVariant.chrom:
                    #childVariant on that position is gt 0/0, but union exists
                    print("error in disjunction on "+str(unionVariant.pos))
                    unionVariant = next(unionVCF, None) 
                else:  
                    #unionVariant on that position is gt 0/0 -> De novo variation
                    if filterPass and next(childVariant.filter.iterkeys()) == 'PASS':   
                        outputVCF.write(childVariant)
                    childVariant = next(childVCF, None)
            while not (childVariant is None or unionVariant is None) and (unionVariant.pos < childVariant.pos):
                if unionVariant.chrom != childVariant.chrom:
                    #unionVariant on that position is gt 0/0 -> De novo variation
                    if filterPass and next(childVariant.filter.iterkeys()) == 'PASS':   
                        outputVCF.write(childVariant)
                    childVariant = next(childVCF, None)
                else:
                    #childVariant on that position is gt 0/0, but union exists
                    print("error in disjunction on "+str(unionVariant.pos))
                    unionVariant = next(unionVCF, None) 
        
        
        if unionVariant is None:
            while not (childVariant is None):
                outputVCF.write(childVariant)
                childVariant = next(childVCF, None)
        elif childVariant is None:
            while not (unionVariant is None):
                print("error in disjunction on "+str(unionVariant.pos))
                unionVariant = next(unionVCF, None)
        else: 
            #variant positions are matched
            unionGT = next(unionVariant.samples.itervalues())['GT']
            childGT = next(childVariant.samples.itervalues())['GT']

            if unionGT == (0,1) and (childGT == (1,1) or childGT == (1,2)) or unionGT == (1,1) and childGT == (1,2):
                # de novo variation on one chromosome
                if childGT == (1,2): childVariant.alts = (childVariant.alts[0],)
                next(childVariant.samples.itervalues())['GT'] = (0,1)
                outputVCF.write(childVariant)
            #elif unionGT == childGT or unionGT == (1,1) and childGT == (0,1):
            elif unionGT == (1,2) and (childGT == (0,1) or childGT == (1,1)):
                print("error in disjunction on "+str(childVariant.pos))

            if chrom != unionVariant.chrom:
                chrom = unionVariant.chrom
                print("disjunction on "+chrom)
    
    outputVCF.close()

In [ ]:
def deNovo():
    findIntersection(childVCF, motherVCF, "outputCM.vcf", True, True, True)
    findIntersection(childVCF, fatherVCF, "outputCF.vcf", True, False, True)
    outputCM = VariantFile("outputCM.vcf")
    outputCF = VariantFile("outputCF.vcf")
    findUnion(outputCM, outputCF, "outputUnion.vcf")
    outputUnion = VariantFile("outputUnion.vcf")
    findDisjunction(outputUnion, childVCF, "outputDeNovo.vcf")

In [ ]:
def deNovoPartial(outputNameCM, outputNameCF):
    outputCM = VariantFile(outputNameCM)
    outputCF = VariantFile(outputNameCF)
    findUnion(outputCM, outputCF, "outputUnion_Partial.vcf")
    outputUnion = VariantFile("outputUnion_Partial.vcf")
    findDisjunction(outputUnion, childVCF, "outputDeNovo_Partial.vcf")

In [ ]:
deNovo()

In [1]:
def analyzeResults(childFileName, deNovoFileNameProject, deNovoFileNamePartial, deNovoFileNameRTG):
    childVCF = VariantFile(childFileName)
    deNovoProjectVCF = VariantFile(deNovoFileNameProject)
    deNovoPartialVCF = VariantFile(deNovoFileNamePartial)
    deNovoRTGVCF = VariantFile(deNovoFileNameRTG)
    countChild = 0
    countChildFiltered = 0
    for rec in childVCF.fetch():
        countChild+=1
        if next(rec.filter.iterkeys()) == 'PASS': countChildFiltered+=1
    print("Total number of variants in child is "+str(countChild))
    print("Number of variants in child that passed filter is "+str(countChildFiltered))
    countDeNovo = 0
    for rec in deNovoProjectVCF.fetch():
        countDeNovo+=1
    print("Number of de novo variants found by this project is "+str(countDeNovo))
    print("Percentage of de novo variants is "+str(countDeNovo*100.0/countChildFiltered))
    
    countDeNovo = 0
    for rec in deNovoPartialVCF.fetch():
        countDeNovo+=1
    print("Number of de novo variants found using RTG Tools intersection is "+str(countDeNovo))
    print("Percentage of de novo variants is "+str(countDeNovo*100.0/countChildFiltered))
    
    countDeNovo = 0
    for rec in deNovoRTGVCF.fetch():
        countDeNovo+=1
    print("Number of de novo variants found by RTG Tools VCFEval is "+str(countDeNovo))
    print("Percentage of de novo variants is "+str(countDeNovo*100.0/countChildFiltered))

In [ ]:
analyzeResults("/sbgenomics/project-files/HG002-NA24385-50x_filtered.vcf", "outputDeNovo.vcf", "outputDeNovo_Partial.vcf", "/sbgenomics/project-files/DeNovo_RTGToolsVCFEval.vcf")